[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [2]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [3]:
# Read dataset with extra params sep='\t', encoding="latin-1"
sep = pd.read_csv('labeledTrainData.tsv','\t', encoding="latin-1")
sep.head()


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [4]:
# stop words
import nltk

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trangphung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from collections import Counter

In [20]:
vocab = Counter()

# Let's apply the example above to count words in our SentimentText
sep.review
for row in sep.review:
    for word in row.split():
        vocab[word] += 1
vocab.most_common(20)

[('the', 287027),
 ('a', 155095),
 ('and', 152662),
 ('of', 142971),
 ('to', 132568),
 ('is', 103227),
 ('in', 85578),
 ('I', 65973),
 ('that', 64555),
 ('this', 57196),
 ('it', 54420),
 ('/><br', 50935),
 ('was', 46697),
 ('as', 42509),
 ('with', 41719),
 ('for', 41067),
 ('but', 33790),
 ('The', 33762),
 ('on', 30765),
 ('movie', 30496)]

In [21]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop' 
# and save them in vocab_reduced
for word, c in vocab.items():
    if not word in stop:
        vocab_reduced[word] = c


vocab_reduced.most_common(20)

[('I', 65973),
 ('/><br', 50935),
 ('The', 33762),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435),
 ('It', 10952),
 ('really', 10814),
 ('even', 10607),
 ('see', 10154),
 ('-', 9355),
 ('get', 8776),
 ('story', 8523),
 ('much', 8507),
 ('time', 7762),
 ('make', 7485)]

In [22]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [23]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

# tokenizer: to break down our twits in individual words
def tokenizer(text):
    token = [] 
    token = text.split()
    
    return token
# stemming: reducing a word to its root
def tokenizer_porter(text):
    token = []
    token = [porter.stem(word) for word in text.split()]
    return []

In [24]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
X = sep.review
y = sep.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [25]:
# Import Pipeline, LogisticRegression, TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop, tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

ValueError: empty vocabulary; perhaps the documents only contain stop words

## 4. Evaluate Model

In [ ]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix


## 5. Export Model 

In [ ]:
# Using pickle to export our trained model
import pickle
import os